# Лабораторная работа №3
## Выполнил студент группы БСТ1903 Талантов О.А.

### Задание №1
Реализовать методы поиска подстроки в строке. 
Добавить возможность ввода строки и подстроки с клавиатуры. 
Предусмотреть возможность существования пробела. 
Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. 
Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.
Алгоритмы:
1.Кнута-Морриса-Пратта
2.Упрощенный Бойера-Мура

In [9]:
import time

# Алгоритм Кнута-Морриса-Пратта
def kmp_search(string, pattern, regsens, spcsens):
    if not regsens:
        string = string.lower()
        pattern = pattern.lower()
    if not spcsens:
        string = string.replace(" ", "")
    j = 0 # Позиция рассматриваемого символа в string
    k = 0 # Позиция рассматриваемого символа в pattern
    while j < len(string):
        if string[j] == pattern[k]:
            j += 1
            k += 1
            if k == len(pattern):
                return(j - k)
        else:
            k = 0
            j += 1
    return(None)
            
# Упрощенный алгоритм Бойера-Мура
def bm_search(string, pattern, regsens, spcsens):
    if not regsens:
        string = string.lower()
        pattern = pattern.lower()
    if not spcsens:
        string = string.replace(" ", "")
    n = len(string)
    m = len(pattern)
    pos = [] # Позиции нужного слова
    bound = 0
    # Формируем таблицу суффиксов (смещения)
    suffshift = [m for i in range(1105)]
    for i in range(0, m-1):
        suffshift[ord(pattern[i])] = m-i-1
    # Поиск подстрок
    # k - проход по string
    # j - проход по pattern
    # i - место начала прохода по s
    i = m-1
    j = k = i
    while j >= 0 and i <= n-1:
        j = m-1
        k = i
        while j >= 0 and string[k] == pattern[j]:
            k -= 1
            j -= 1
        i += suffshift[ord(string[i])]
    if j <= 0:# нашли
        return(k+1)
    return(None) # не нашли
            
option = int(input("Ввод строк: 0 - вручную; 1 - готовый пример. "))
regsens = bool(int(input("Поиск чувствителен к регистру? (0/1). ")))
spcsens = bool(int(input("Поиск чувствителен к пробелам? (0/1). ")))
if option == 0:
    string = input("Введите строку:")
    pattern = input("Введите искомую подстроку:")
if option == 1:
    string = "an d And style never met and those among great. At wander no or september sportsmen he perfectly happiness attending."
    pattern = "and"
print("string =", string, "\npattern =", pattern)
    
# Алгоритм Кнута-Морриса-Пратта
start_time = time.time_ns()
print("[kmp_search] found pattern at:", kmp_search(string, pattern, regsens, spcsens))
print("[kmp_search] took %s nanoseconds" % (time.time_ns() - start_time))
# Упрощенный алгоритм Бойера-Мура
start_time = time.time_ns()
print("[bm_search] found pattern at:", bm_search(string, pattern, regsens, spcsens))
print("[bm_search] took %s nanoseconds" % (time.time_ns() - start_time))
# Стандартная функция поиска
if not regsens:
    string = string.lower()
    pattern = pattern.lower()
if not spcsens:
    string = string.replace(" ", "")
start_time = time.time_ns()
print("[std_search] found pattern at:", string.find(pattern))
print("[std_search] took %s nanoseconds" % (time.time_ns() - start_time))

Ввод строк: 0 - вручную; 1 - готовый пример. 1
Поиск чувствителен к регистру? (0/1). 0
Поиск чувствителен к пробелам? (0/1). 1
string = an d And style never met and those among great. At wander no or september sportsmen he perfectly happiness attending. 
pattern = and
[kmp_search] found pattern at: 5
[kmp_search] took 0 nanoseconds
[bm_search] found pattern at: 5
[bm_search] took 995200 nanoseconds
[std_search] found pattern at: 5
[std_search] took 0 nanoseconds


### Задание 2 
«Пятнашки»
Написать программу, определяющую, является ли данное расположение «решаемым», то есть можно ли из него за конечное число шагов перейти к правильному. Если это возможно, то необходимо найти хотя бы одно решение - последовательность движений, после которой числа будут расположены в правильном порядке.

In [13]:
import queue
import time

# Инверсия - когда больший элемент стоит до меньшего элемента
#Считаем для всех костяшек
def count_inversions(array):
    inversions = 0
    size = len(array)
    for i in range(0, size-1):
        for j in range(i+1, size):
            if array[i] > 0 and array[j] > 0:
                if array[i] > array[j]:
                    inversions += 1
    return inversions

# Для определения решаемы ли костяшки
def is_solvable(array, m=4, n=4):
    inversions = count_inversions(array)
    size = len(array)
    # В случае чётного количества костяшек смотрим, на какой строке снизу
    #находится пустое место (последняя строка - нечётное, предпоследняя - чётное и т.д.
    #для решаемости необходима разная чётность кол-ва инверсий и строки пустого места (чёт/нечёт)
    if n*m%2==0:
        for i in range(0, size):
            if array[i] == 0:
                if ((m-i//n)%2 == 1):
                    return(inversions%2 == 0)
                else:
                    return(inversions%2 != 0)
        return False
    # В случае нечётного количества костяшек нужно, чтобы количество инверсий было чётно
    if inversions%2==0:
        return True
    return False
    
# Класс состояния поля пятнашек
class BoardState:
    array = [] # Для хранения расположение костяшек
    n = 4 # Размерность поля
    m = 4
    pos0 = 0 # Для хранения позиции пустого места
    moves = [] # Для хранения необходимых перемещения для его достижения
    
    def __init__(self, array, n=4, m=4, pos0 = 15, moves=[]):
        self.array = array
        self.n = n
        self.m = m
        self.pos0 = pos0
        self.moves = moves
        
    # Переопределение сравнения по стоимости (законченности поля)
    def __lt__(self, other):
        return self.get_total_mdistance() < other.get_total_mdistance()
    def __le__(self, other):
        return self.get_total_mdistance() <= other.get_total_mdistance()
        
    # Для проверки равенства игровых полей, проверяем размерность и костяшки
    def equals(self, board):
        return (self.n == board.n and self.m == board.m and self.array == board.array)
    
    # Для подсчёта костяшек вне позиции
    def get_outofpos_count(self):
        counter = 0
        for i in range(0,len(self.array)):
            # Если проверяем не пустое место и не совпадает позиция
            if (self.array[i] != 0 and i+1 != self.array[i]):
                counter += 1
        return counter
    
    # Manhattan Distance - это расстояние на которое нужно передвинуть
    #костяшку, чтобы она оказалась в позиции. В данном методе мы вычисляем
    #сумму всех расстояний всех костяшек поля.
    def get_total_mdistance(self):
        distance = 0
        for i in range(0,len(self.array)):
            # Если проверяем не пустое место вычисляем расстояние
            elem = self.array[i]
            if elem != 0:
                x = abs((elem-1)%self.n - i%self.n)
                y = abs((elem-1)//self.n - i//self.n)
                distance += x
                distance += y
        return distance
        
    # Для возвращения нового состояния при передвижении костяшки
    def get_moved_state(self, direction):
        # Проверяем направление
        moved_index = None
        if direction == "up":
            moved_index = self.pos0+self.n
        if direction == "down":
            moved_index = self.pos0-self.n
        if direction == "left":
            moved_index = self.pos0+1
        if direction == "right":
            moved_index = self.pos0-1  
        # Если указано правильное направление
        if moved_index != None:
            # Добавление нового хода
            new_moves = self.moves.copy()
            new_moves.append(self.array[moved_index])
            # Замена костяшки и пустого места
            new_array = self.array.copy()
            new_array[self.pos0] = new_array[moved_index]
            new_array[moved_index] = 0
            return BoardState(new_array, self.n, self.m, moved_index, new_moves)
        else:
            print("BoardState.get_moved_state(): Invalid direction, use up/down/left/right\n")
            return 0 
    
def solve_astar(array, n=4, m=4, goal=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0]):
    # Если нерешаемо, возвращаем пустой массив
    if not is_solvable(array):
        return "Not solvable"
    else:
        # Создаём приоритетную очередь, для обработки состояний поля
        pq = queue.PriorityQueue()
        current_board = BoardState(array, n, m, array.index(0))
        # Добавляем в очередь изначальное положение игрового поля
        pq.put(current_board)
        # Два массива для доступа к ещё необработанным и уже обработанным состояниям
        open_boardstates = []
        closed_boardstates = []
        # Пока не найдём нужное поле
        while current_board.array != goal and not pq.empty():
            closed_boardstates.append(current_board)
            # Добавляем в очередь все возможные состояния достижимые из данного
            if current_board.pos0//n != m-1:
                next_board = current_board.get_moved_state("up")
                if not any(x.array == next_board.array for x in closed_boardstates):
                    pq.put(next_board)
            if current_board.pos0//n != 0:
                next_board = current_board.get_moved_state("down")
                if not any(x.array == next_board.array for x in closed_boardstates):
                    pq.put(next_board)
            if current_board.pos0%n != n-1:
                next_board = current_board.get_moved_state("left")
                if not any(x.array == next_board.array for x in closed_boardstates):
                    pq.put(next_board)
            if current_board.pos0%n != 0:
                next_board = current_board.get_moved_state("right")
                if not any(x.array == next_board.array for x in closed_boardstates):
                    pq.put(next_board)
            current_board = pq.get()
        if pq.empty():
            print("Did not found")
        return current_board.moves;
    
# Проверка на работу граничных случаев     
print("moves to solve 1st:", solve_astar([1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0])) #Решаемо
print("moves to solve 2nd:", solve_astar([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0])) #Решаемо
print("moves to solve 3rd:", solve_astar([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14, 0])) #Нерешаемо

# Вывод сложного паззла
start_time = time.time()
moves = solve_astar([9, 5, 14, 15, 13, 6, 7, 11, 12, 10, 8, 2, 3, 4, 1, 0])
print("[solve_astar] moves: ", moves, "\n[solve_astar] total moves:", len(moves)) #Решаемо
print("[solve_astar] took %s seconds" % (time.time() - start_time))


moves to solve 1st: [15, 14, 10, 13, 9, 10, 14, 15]
moves to solve 2nd: []
moves to solve 3rd: Not solvable
[solve_astar] moves:  [2, 8, 1, 4, 3, 12, 13, 9, 5, 14, 15, 11, 8, 2, 4, 3, 12, 13, 9, 5, 14, 15, 11, 8, 2, 4, 3, 12, 10, 1, 12, 3, 4, 12, 7, 2, 8, 11, 15, 6, 2, 15, 6, 2, 1, 7, 15, 6, 2, 1, 7, 10, 3, 15, 6, 7, 5, 14, 1, 2, 11, 8, 12, 4, 15, 3, 10, 6, 3, 15, 4, 12, 8, 11, 7, 3, 6, 10, 13, 9, 14, 5, 10, 14, 9, 13, 14, 10, 3, 7, 11, 8, 7, 11, 2, 3, 10, 6, 11, 7, 8, 2, 3, 10, 6, 11, 12, 4, 15, 12, 11, 6, 10, 3, 2, 8, 4, 11, 12, 15, 11, 12, 6, 10, 3, 2, 8, 4, 7, 8, 2, 3, 10, 6, 8, 7, 12, 8, 6, 10, 3, 2, 7, 3, 10, 6, 8, 12, 3, 7, 4, 3, 12, 8, 6, 10, 7, 6, 8, 12, 3, 4, 6, 3, 12, 8, 3, 7, 2, 6, 7, 3, 8, 12, 3, 8, 12, 3, 8, 7, 6, 2, 7, 6, 4, 8, 3, 12, 6, 3, 8, 4, 3, 7, 10, 6, 15, 11, 12, 8, 7, 10, 6, 15, 10, 7, 8, 12, 11, 14, 15, 10, 12, 11, 14, 12, 11, 14, 12, 15, 10, 11, 14, 12, 15, 14, 11, 10, 14, 15] 
[solve_astar] total moves: 230
[solve_astar] took 2.1938254833221436 seconds


### Вывод
В ходе лабораторной работы мы научились искать подстроку в строке, используя алгоритм Кнута-Морриса-Пратта и упрощенный алгоритм Бойера-Мурк, а также реализовали возможность ввода строки и подстроки с клавиатуры и возможность выбора опции чувствительности или нечувствительности к регистру, предусмотрели возможность существования пробела, а также сравнили с работой встроенного поиска. Также реализовали известную игру "Пятнашки". 